#Купил значит я себе рахат-лукум, как в Греции

In [3]:
!pip install catboost --break-system-packages # Теперь я вич инфецированный
!pip install unrar --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [19]:
import numpy as np
import pandas as pd
import math
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

In [ ]:
!unzip /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip

unzip:  cannot find or open /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip, /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip.zip or /content/drive/MyDrive/Попытки-в-Catboost/new-york-city-taxi-fare-prediction.zip.ZIP.


In [7]:
# Функция для расчета расстояния между координатами
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Радиус Земли в километрах
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    return R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))

In [8]:
# Загрузка данных
data = pd.read_csv('/home/dark516/Desktop/vsosii_kolya/AI_code_vseross_AI/new-york-city-taxi-fare-prediction/train.csv', nrows=1000000)
data = data.dropna() # Убираем пустые значения

In [9]:
# Преобразование даты и времени
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], errors='coerce')
data = data.dropna(subset=['pickup_datetime'])

In [10]:
# Создание временных признаков
data['year'] = data['pickup_datetime'].dt.year
data['month'] = data['pickup_datetime'].dt.month
data['day'] = data['pickup_datetime'].dt.day
data['day_of_week'] = data['pickup_datetime'].dt.dayofweek
data['hour'] = data['pickup_datetime'].dt.hour

In [11]:
# Создание признака расстояния
data['distance'] = haversine_distance(data['pickup_latitude'], data['pickup_longitude'],
                                      data['dropoff_latitude'], data['dropoff_longitude'])

In [12]:
features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
            'passenger_count', 'year', 'month', 'day', 'day_of_week', 'hour', 'distance']
target = 'fare_amount'

X = data[features]
y = data[target]

# Разделение данных
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# Создание и обучение модели
model = CatBoostRegressor(iterations=1100, learning_rate=0.1, depth=10, loss_function='RMSE', verbose=100)
model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)

0:	learn: 9.1208654	test: 9.1393277	best: 9.1393277 (0)	total: 74.6ms	remaining: 1m 21s
100:	learn: 4.1874807	test: 4.4096926	best: 4.4096926 (100)	total: 4.41s	remaining: 43.6s
200:	learn: 3.9535158	test: 4.3429366	best: 4.3429366 (200)	total: 8.69s	remaining: 38.8s
300:	learn: 3.8136933	test: 4.3115999	best: 4.3115999 (300)	total: 12.9s	remaining: 34.2s
400:	learn: 3.7036795	test: 4.3013155	best: 4.3012392 (399)	total: 17.2s	remaining: 30s
500:	learn: 3.6188206	test: 4.2971905	best: 4.2965746 (499)	total: 21.4s	remaining: 25.6s
600:	learn: 3.5400298	test: 4.2918706	best: 4.2917266 (599)	total: 25.6s	remaining: 21.2s
700:	learn: 3.4792678	test: 4.2879476	best: 4.2878324 (687)	total: 29.8s	remaining: 17s
800:	learn: 3.4277746	test: 4.2882074	best: 4.2869488 (749)	total: 34.1s	remaining: 12.7s
900:	learn: 3.3827338	test: 4.2877059	best: 4.2869488 (749)	total: 38.3s	remaining: 8.46s
1000:	learn: 3.3409568	test: 4.2882746	best: 4.2869488 (749)	total: 42.5s	remaining: 4.21s
1099:	learn: 3.

In [22]:
# Оценка модели
y_pred = model.predict(X_val)
rmse = math.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE на тестовой выборке: {rmse:.2f}")

RMSE на тестовой выборке: 4.29


In [23]:
# Загрузка тестовых данных
test_data = pd.read_csv('new-york-city-taxi-fare-prediction/test.csv')

# Предобработка тестовых данных
test_data['pickup_datetime'] = pd.to_datetime(test_data['pickup_datetime'], errors='coerce')
test_data['year'] = test_data['pickup_datetime'].dt.year
test_data['month'] = test_data['pickup_datetime'].dt.month
test_data['day'] = test_data['pickup_datetime'].dt.day
test_data['day_of_week'] = test_data['pickup_datetime'].dt.dayofweek
test_data['hour'] = test_data['pickup_datetime'].dt.hour

test_data['distance'] = haversine_distance(test_data['pickup_latitude'], test_data['pickup_longitude'],
                                      test_data['dropoff_latitude'], test_data['dropoff_longitude'])

In [24]:
predictions = model.predict(test_data[features]) # ТО САМОЕ ПРЕДСКАЗАНИЕ

# Создаем DataFrame для отправки
submission = pd.DataFrame({
    'key': test_data['key'],
    'fare_amount': predictions
    })

# Сохранение результатов
submission.to_csv('taxi_fare_submission.csv', index=False)
print("Файл taxi_fare_submission.csv успешно создан.")

Файл taxi_fare_submission.csv успешно создан.
